## Experimentation.ipynb

This file creates custom mmsegmentation config files, trains models, and logs them to wandb (Weights and Biases).

Thanks to Bradley for putting this together

In [ ]:
# Weights and Biases 
import wandb
wandb.login()

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
import os
import torch, torchvision
import mmseg
import mmcv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mmcv import Config
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
from mmseg.apis import set_random_seed
from mmseg.utils import get_device
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

import os.path as osp
import numpy as np
import cv2

import numpy as np
from PIL import Image

import datetime


In [ ]:
use_wandb = True
create_config = False
create_dir = True
custom_backbone = True
custom_head = True


base_config="fcn_head_generic.py"

cfg = Config.fromfile('configs/' + base_config)
data_root="data/cityscapes"

cfg.num_classes = 12 # Camvid: 11 + 1 void
cfg.dataset_type = "Cityscapes11Dataset"
cfg.data_root = data_root

cfg.batch_size = 4
cfg.workers_per_gpu = 1

cfg.optimizer.lr=0.045
cfg.optimizer.momentum=0.9

cfg.runner.type = "EpochBasedRunner" 
cfg.runner.max_epochs = 500
cfg.runner.max_iters = None
cfg.log_config.interval = 1
cfg.evaluation.interval = 1
cfg.checkpoint_config.interval = 100

cfg.evaluation.save_best="mIoU"
cfg.evaluation.by_epoch = True
cfg.checkpoint_config.by_epoch = True
cfg.lr_config.by_epoch = True

# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True, momentum=0.01)
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg

cfg.crop_size = (480, 640) # original // 1.5

model_comment = "Cityscapes. ResNet34, 2 stages"

# Backbone config
if custom_backbone:
    cfg.model.backbone.type='ResNet'
    cfg.model.backbone.depth=34
    cfg.model.backbone.pretrained='open-mmlab://resnet34'
    cfg.model.backbone.num_stages=2
    cfg.model.backbone.strides=(1,2)
    cfg.model.backbone.dilations=(1,1)
    cfg.model.backbone.out_indices=(0,1)
    cfg.model.backbone.contract_dilation=True
# Decode head config
if custom_head:
    cfg.model.decode_head.type="FCNHead"
    cfg.model.backbone.pretrained=None
    cfg.model.decode_head.in_channels=128
    cfg.model.decode_head.in_index=-1
    cfg.model.decode_head.channels=64
    cfg.model.decode_head.num_classes=cfg.num_classes
    cfg.model.decode_head.kernel_size=3
    cfg.model.decode_head.dropout_ratio=0.45


cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(720, 960), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(720, 960),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.samples_per_gpu = cfg.batch_size
cfg.data.workers_per_gpu = cfg.workers_per_gpu

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = "leftImg8bit/train"
cfg.data.train.ann_dir = "gtFine/train"
cfg.data.train.pipeline = cfg.train_pipeline

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = "leftImg8bit/val"
cfg.data.val.ann_dir = "gtFine/val"
cfg.data.val.pipeline = cfg.test_pipeline

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = "leftImg8bit/val"
cfg.data.test.ann_dir = "gtFine/val"
cfg.data.test.pipeline = cfg.test_pipeline


# Using pre-trained segmentation model
# cfg.load_from = 'work_dirs/experimentation.pth'

is_pretrained = False

if ("pretrained" in cfg.model.backbone):
    if cfg.model.backbone["pretrained"] is not None:
        is_pretrained = True
if ("pretrained" in cfg.model):
    if cfg.model["pretrained"] is not None:
        is_pretrained = True


# Create config name based on whether weights (backbone) are trained from scratch
if is_pretrained:
    config_filename = str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")) + "_" + \
        str(cfg.dataset_type) + "_" + \
        str(cfg.model.decode_head.type) + "_" + \
        str(cfg.model.backbone.type) + ".py"
else:
    config_filename = str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")) + "_" + \
        str(cfg.dataset_type) + "_" + \
        str(cfg.model.decode_head.type) + "_" + \
        str(cfg.model.backbone.type) + "_scratch" + ".py"

    
if create_dir:
    # Set up working dir to save files and logs.
    cfg.work_dir = './work_dirs/experimentation/' + config_filename[:-3]
    # Create work_dir
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))


# Set seed for result reproduction
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()


wandb_cfg_dict = {
    "learning_rate": cfg.optimizer.lr,
    "momentum": cfg.optimizer.momentum,
    "batch_size": cfg.batch_size,
    "backbone": cfg.model.backbone.type,
    "decodehead": cfg.model.decode_head.type,
    "dataset": cfg.dataset_type,
    "epochs": cfg.runner.max_epochs,
    "is_pretrained": is_pretrained,
    "comment":model_comment
}

cfg.log_config.hooks[0].by_epoch=True

# Log to WandB
if use_wandb:
    cfg.log_config.hooks.append(
        dict(type='MMSegWandbHook', by_epoch=True, # The Wandb logger is also supported, It requires `wandb` to be installed.
                init_kwargs={'entity': "austin-bevac", # The entity used to log on Wandb
                            'project': "2023-honours-austin", # Project name in WandB
                            'config': wandb_cfg_dict,
                            'name': config_filename[:-3]})
    )

# Create config file
if create_config:
    cfg.dump("configs/" + str(config_filename))


In [ ]:
cfg = Config.fromfile("./configs/2023-04-07-23-37-11_Camvid11Dataset_DepthwiseSeparableFCNHead_ResNet.py")

In [ ]:
model = build_segmentor(cfg.model)
datasets = [build_dataset(cfg.data.train)]

In [ ]:
from torchsummary import summary
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)
print(model)


In [ ]:
train_segmentor(model, datasets, cfg, distributed=False, validate=True, meta=dict())

In [ ]:
# Manual config file creation 
cfg.dump("configs/" + str(config_filename))